In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image as im
import matplotlib.image as mpimg
from tqdm.notebook import tqdm
from glob import glob
import os
import cv2
from PIL import Image
ff = 10000000
f = 100

def vis(img): #reorders array to be compatible with matplotlib
    im = np.moveaxis(img,0, -1)
    plt.imshow(im)
    plt.show()


def numpy_to_image(img, path): #converts numpy array to image
    if not os.path.exists(f'{path}.png'):
        img = np.moveaxis(img,0, -1)
        img = np.dot(img[...,:3], [0.299, 0.587, 0.114]) 
        mpimg.imsave(f'{path}.png', img, cmap='gray')
        #Image.fromarray(img).save(f'{path}.png')
        return 0,0
    else:
        img = np.moveaxis(img,0, -1)
        print(f'{path}.png')
        b="/".join(path.split('/')[0:-2])
        bb = int(path.split('/')[-1])+1
        img = np.dot(img[...,:3], [0.299, 0.587, 0.114]) 
        mpimg.imsave(f"""{b}{bb}.png""", img,cmap='gray')
        #Image.fromarray(img).save(f"""{b}{bb}.png""")
        return 1,0
def binarize(i):
    return 1 if i > 0 else 0

# name = test_mask(f, False)
#print(name)

test_data = np.load(f'/home/smoothjazzuser/Desktop/fast/NORMAL-TRAIN/ep-0000/00000000.npz', allow_pickle=True)
test_data.files
test_data['state'].shape
tr = "/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames/"
te = "/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames/"
#
#glob('/home/smoothjazzuser/Desktop/fast/TEST/*')
#glob('/home/smoothjazzuser/Desktop/fast/TEST/BlackScreen/*/*.npz')

In [2]:
#make sure each dataset contains multiples of five images
def check_dataset(path):
    for i in glob(path + '/*'):
        files = glob(i + '/*')
        num = len(files)
        div = num/5
        if div % 1 != 0:
            print(i, num, div)
        

check_dataset(tr)
check_dataset(te)

In [3]:
#sorted(glob.glob(f"/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames/54/*.png"), key=lambda x: int(x.split('/')[-1].split('.')[0]))

masked_list = []
folder_count = 0
# for every file in each bug type folder, save the image to the corresponding folder under MNAD
with tqdm(total=len(glob('/home/smoothjazzuser/Desktop/fast/TEST/*'))) as t:
    for i in glob('/home/smoothjazzuser/Desktop/fast/TEST/*'):
        
        ff = glob(f'{i}/*')
        ff.sort()

        for ii in ff:
            
            counter = 0
            folder_count+=1
            if len(str(folder_count)) >= 2:
                    f_name = '' + str(folder_count)
            else:
                f_name = '0' + str(folder_count)

            images = glob(f'{ii}/*.npz')
            images.sort()

            for j in images:
                
                if len(str(counter)) >= 3:
                    name = str(counter)
                elif len(str(counter)) == 2:
                    name = '0' + str(counter)
                else:
                    name = '00' + str(counter)
                
                data = np.load(j, allow_pickle=True)
                masked_list.append(binarize(np.sum(data['info'][1])))
                if not os.path.exists(f"{te + f_name}"):
                    os.makedirs(f"{te + f_name}")
                c, f =numpy_to_image(data['state'], te + f_name + "/" + name)
                counter += c
                folder_count += f
                counter+=1
            
    t.update(1)

np.save('frame_labels_bugs.npy', np.array(np.expand_dims(np.array(masked_list), axis=0)))

  0%|          | 0/10 [00:00<?, ?it/s]

In [4]:
del masked_list
import gc
gc.collect()

706

In [5]:
folder_count = 0
for i in tqdm(glob('/home/smoothjazzuser/Desktop/fast/NORMAL-TRAIN/*')):

    folder_count+=1
    if len(str(folder_count)) >= 2:
        f_name = '' + str(folder_count)
    else:
        f_name = '0' + str(folder_count)
    counter = -1
    if not os.path.exists(f"{tr + f_name}"):
            os.makedirs(f"{tr + f_name}")
    
    images = glob(f'{i}/*.npz')
    images.sort()

    for j in images:
        counter+=1
        if len(str(counter)) >= 3:
            name = str(counter)
        elif len(str(counter)) == 2:
            name = '0' + str(counter)
        else:
            name = '00' + str(counter)
                
        data = np.load(j, allow_pickle=True)
        #masked_list.append(np.sum(data['info'][1]))
        numpy_to_image(data['state'], tr + f_name + "/" + name)
        
    

  0%|          | 0/70 [00:00<?, ?it/s]